# 20220905 | Decision Trees | Naïve Bayes

In [6]:
from math import prod

import numpy as np
import pandas as pd

import yaml

## The Dataset

In [7]:
def I(*args):
    total = sum(args)
    ans = 0
    for arg in args:
        ans -= np.log2(arg / total) * (arg / total)
        # print(f"- {arg}/{total}*log2({arg}/{total})", end=" ")
    # print(f"= {ans}")
    return ans

In [8]:
age = {
    1: "<=30",
    2: "31..40",
    3: ">40"
}
income = {
    1: "high",
    2: "medium",
    3: "low"
}
credit_rating = {
    1: "fair",
    2: "excellent"
}

df = pd.DataFrame({
    "age": [1, 1, 2, 3, 3, 3, 2, 1, 1, 3, 1, 2, 2, 3],
    "income": [1, 1, 1, 2, 3, 3, 3, 2, 3, 2, 2, 2, 1, 2],
    "student": [False, False, False, False, True, True, True, False, True, True, True, False, True, False],
    "credit_rating": [1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 2, 2, 1, 2],
    "buys_computer": [False, False, True, True, True, False, True, False, True, True, True, True, True, False]
})
df

,age,income,student,credit_rating,buys_computer
0,1,1,False,1,False
1,1,1,False,2,False
2,2,1,False,1,True
3,3,2,False,1,True
4,3,3,True,1,True
5,3,3,True,2,False
6,2,3,True,2,True
7,1,2,False,1,False
8,1,3,True,1,True
9,3,2,True,1,True


## Decision Tree
### Some Functions Needed

In [9]:
def get_val_counts(df):
    y_vals = df.iloc[:, -1].value_counts().to_dict()
    x_vals = {attr: df[attr].value_counts().to_dict() for attr in df.iloc[:, :-1]}
    return x_vals, y_vals

In [10]:
def print_yaml(dict):
    print(yaml.dump(dict, default_flow_style=False))

### Formulas

$entropy = info(D) = -\sum(p_ilog_2(p_i))$

$information = info_A(D) = \sum_{j=1}^{v}(\frac{|D_j|}{|D|}\cdot I(D_j))$

$information\ gained = gain(A) = info(D) - info_A(D)$

### Implementation

In [11]:
# finding the gain for the given df
def gains(df):
    y = df.iloc[:, -1]
    x_vals, y_vals = get_val_counts(df)

    total = y.count()
    entropy = I(*[y_vals[y] for y in y_vals])
    x_info = {}
    for x in x_vals: # for each attribute
        x_info[x] = 0 # the information gain is set to 0
        for attr in x_vals[x]: # for each possible value of attribute x
            val = x_vals[x][attr] # the number of records with that value
            y_for_x = df.loc[df[x] == attr].iloc[:, -1].value_counts().to_list() # the class label distribution for each record with that value
            x_info[x] += (val / total) * I(*y_for_x)
    x_gain = {x: (entropy - x_info[x]) for x in x_info}
    return x_gain

In [12]:
def make_decision_tree(df):
    # getting all the values of x and y for the given node
    x_vals, y_vals = get_val_counts(df)

    # if there is only one class label left
    if len(y_vals) == 1:
        class_label = list(y_vals.keys())[0]
        return {'class': class_label}

    # if we need to calculate further
    else:
        gain = gains(df) # get the gain for each attribute
        x_max = max(gain, key=lambda x: gain[x]) # get the attribute with the max gain
        x_branches = [
            {
                val: make_decision_tree(df.loc[df[x_max] == val])
            } for val in x_vals[x_max]
        ]
        return {x_max: x_branches}

print_yaml(make_decision_tree(df))
print(make_decision_tree(df))

age:
- 1:
    student:
    - false:
        class: false
    - true:
        class: true
- 3:
    credit_rating:
    - 1:
        class: true
    - 2:
        class: false
- 2:
    class: true

{'age': [{1: {'student': [{False: {'class': False}}, {True: {'class': True}}]}}, {3: {'credit_rating': [{1: {'class': True}}, {2: {'class': False}}]}}, {2: {'class': True}}]}


## Naïve Bayes

In [13]:
X = {
    'age': 1,
    'income': 2,
    'student': True,
    'credit_rating': 1
}

In [14]:
def naive_bayes(df, sample):
    x_vals, y_vals = get_val_counts(df)
    y_total = sum(y_vals[y] for y in y_vals)
    pc = {y: y_vals[y]/y_total for y in y_vals}
    pxc = {
        attr: { # for every class
        # for all class labels, calclulate P(X|Ci)
            # y: (the number of records with the attr value given a class label y) / (no of records with class label y)
            y: len(df.loc[(
                df[attr] == sample[attr]
            ) & (
                df.iloc[:, -1] == y
            )]) / y_vals[y] for y in y_vals
        } for attr in sample
    }

    # P(X | Ci) * P(Ci) for each class label
    finals = {
        y: prod([pxc[attr][y] for attr in pxc]) * pc[y] for y in y_vals
    }

    # return the maximum
    return max(finals, key=lambda x: finals[x])

In [15]:
naive_bayes(df, X)

True

## Another Dataset

In [17]:
df2 = pd.read_csv("../iris.csv")
df2

,sepallength,sepalwidth,petallength,petalwidth,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [19]:
print_yaml(make_decision_tree(df2))

petallength:
- 1.5:
    class: Iris-setosa
- 1.4:
    class: Iris-setosa
- 5.1:
    sepallength:
    - 5.8:
        class: Iris-virginica
    - 6.0:
        class: Iris-versicolor
    - 6.5:
        class: Iris-virginica
    - 6.3:
        class: Iris-virginica
    - 6.9:
        class: Iris-virginica
    - 5.9:
        class: Iris-virginica
- 4.5:
    sepallength:
    - 6.0:
        class: Iris-versicolor
    - 6.4:
        class: Iris-versicolor
    - 5.7:
        class: Iris-versicolor
    - 5.6:
        class: Iris-versicolor
    - 6.2:
        class: Iris-versicolor
    - 5.4:
        class: Iris-versicolor
    - 4.9:
        class: Iris-virginica
- 1.6:
    class: Iris-setosa
- 1.3:
    class: Iris-setosa
- 5.6:
    class: Iris-virginica
- 4.7:
    class: Iris-versicolor
- 4.9:
    sepalwidth:
    - 3.1:
        class: Iris-versicolor
    - 2.5:
        class: Iris-versicolor
    - 2.8:
        class: Iris-virginica
    - 2.7:
        class: Iris-virginica
    - 3.0:
        clas